## Experiment Setup

### Random seed / PyTorch / CUDA related

In [1]:
import time
import datetime
import os
import sys
import itertools

# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')
    
    # If there's a package I need to install separately, do it here
    !pip install pyro-ppl

    # cd to the appropriate working directory under my Google Drive
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    
    # List the directory contents
    !ls

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 682

### Third party libraries (NumPy, PyTorch, Pyro)

In [2]:
# Third party libraries import
import numpy as np
import torch
import pyro
import matplotlib.pyplot as plt

# Print version information
print("Python Version: " + sys.version)
print("NumPy Version: " + np.__version__)
print("PyTorch Version: " + torch.__version__)
print("Pyro Version: " + pyro.__version__)

Python Version: 3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
NumPy Version: 1.17.2
PyTorch Version: 1.3.1
Pyro Version: 1.0.0


In [3]:
# More imports...
from torch import nn, optim
from torch.utils.data import random_split, DataLoader, RandomSampler
import torchvision
import torchvision.transforms as transforms
from pyro.infer import SVI, Trace_ELBO, HMC, MCMC

# Import model and dataset classes from ronald_bdl
from ronald_bdl import models, datasets
from ronald_bdl.models import utils

# pyplot setting
%matplotlib inline

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')

    torch.backends.cudnn.deterministic = True
    
    # Disable 'benchmark' mode
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    use_pin_memory = True # Faster Host to GPU copies with page-locked memory

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))
else:
    torch_device = torch.device('cpu')
    use_pin_memory = False

### Variable settings

#### Data prep

In [4]:
# CIFAR10 data transformation setting
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Set the proportion of the original dataset to be available as a whole
subset_proportions = [0.01, 0.1, 1]

# Proportion of the dataset to be used for training
dataset_train_size = 0.8

# Number of dataset splits
n_splits = 10

#### NN settings

In [5]:
# Dropout
dropout_rates = [0.1, 0.3, 0.5]

# Length-scale
length_scale_values = [1e-2]

# Model Precision
tau_values = [0.1, 0.15, 0.2]


### Training setup

In [6]:
# Epochs
n_epoch_values = [40, 400, 4000]

# Optimizer learning rate
optimizer_learning_rate = 0.001 # PyTorch default value is 0.001

# Data batch sizes
n_training_batch = 512

# Number of test predictions (for each data point)
n_prediction = 500

# Cross Entropy to minimize
objective = nn.CrossEntropyLoss()

# Test start time
test_start_time = datetime.datetime.today().strftime('%Y%m%d%H%M')

print(test_start_time)

201912031234


## Train the network

In [ ]:
for subset_prop in subset_proportions:
    
    """
    Dataset multiple splits prep
    """
    # Prepare new subset of the original dataset
    subset = datasets.CIFAR10(
        root='./datasets_files', limit_size=subset_prop, transform=transform, download=True)

    # Determine sizes of training and testing set
    train_size = int(dataset_train_size * len(subset))
    test_size = len(subset) - train_size

    # Print the size of the subset
    print("subset size = " + str(subset.data.shape))
    print("training set size = %d" % train_size)
    print("test set size = %d" % test_size)
    print()

    # Prepare multiple sets of random train-test splits 
    # to test the parameter combination
    subset_splits = []

    for _ in range(n_splits):
        train, test = random_split(subset, lengths=[train_size, test_size])
        subset_splits.append((train, test))

    # With all the splits, test out each combination of hyperparameters
    for dropout_rate, length_scale, tau in itertools.product(
        dropout_rates, length_scale_values, tau_values,
    ):  
        # Reset the random number generator for each method (to produce identical results)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)
        pyro.set_rng_seed(random_seed)

        # Print parameter combinations being tested
        print(
            "subset %f, dropout_rate %f, length_scale %f, tau %f"
            % (subset_prop, dropout_rate, length_scale, tau))

        """
        Training & testing
        """

        # Try learning with different splits
        for s, (train, test) in enumerate(subset_splits):
            train_loader = DataLoader(train, batch_size=n_training_batch, pin_memory=use_pin_memory)

            # Prepare network
            network = models.SimpleCIFAR10(
                dropout_rate=dropout_rate,
                dropout_type='bernoulli',
            )

            # Send the whole model to the selected torch.device
            network.to(torch_device)

            # Model to train mode
            network.train()

            # Adam optimizer
            # https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam

            # NOTE: Need to set L2 regularization from here
            reg_strength = utils.reg_strength(dropout_rate, length_scale, train_size, tau)

            print('reg_strength = ' + str(reg_strength))

            optimizer = optim.Adam(
                network.parameters(),
                lr=optimizer_learning_rate,
                weight_decay=reg_strength, # L2 regularization
            )

            accumulated_epochs = 0

            for n_epoch in n_epoch_values:

                """
                Training
                """

                print('Training with split %d' % s)
                print("Training for %d epochs total." % n_epoch)
            
                # Record training start time (for this split)
                tic = time.time()

                for epoch in range(n_epoch-accumulated_epochs): # loop over the dataset multiple times
                    # Mini-batches
                    for data in train_loader:
                        # get the inputs; data is a list of [inputs, labels]
                        inputs, targets = data

                        # Store the batch to torch_device's memory
                        inputs = inputs.to(torch_device)
                        targets = targets.to(torch_device)

                        # zero the parameter gradients
                        optimizer.zero_grad()

                        # forward + backward + optimize
                        outputs = network(inputs)

                        loss = objective(outputs, targets)
                        loss.backward()

                        optimizer.step()

                    print("epoch %d loss = %f" % (epoch, loss.item()))

                # Record training end time
                toc = time.time()
                
                # Track the number of epochs done so far
                accumulated_epochs += (n_epoch - accumulated_epochs)
                
                # Report the total training time
                print("training time = " + str(toc - tic) + " seconds")
                
                # Report the final loss
                print("final loss = %f" % (loss.item()))
                print()

                """
                Testing
                """

                # Model to eval mode
                network.eval()

                # Store the batch to torch_device's memory
                test_loader = DataLoader(test, batch_size=n_training_batch, pin_memory=use_pin_memory)

                # Record testing start time
                tic_testing = time.time()

                _, mean, metrics = network.predict_dist(test_loader, n_prediction)

                # Record testing end time
                toc_testing = time.time()
                
                # Report the total testing time
                print("testing time = " + str(toc_testing - tic_testing) + " seconds")

                # Record all the scores to the score files
                """
                Results file storage
                """

                # Create directory to store results for the current test configuration
                test_results_path = os.path.join(
                    './test_results',
                    'error_convergence_2',
                    'CIFAR-10',
                    test_start_time,
                    (
                        str(subset_prop)
                        + '_' + str(dropout_rate) 
                        + '_' + str(length_scale)
                        + '_' + str(tau)
                        + '_' + str(n_epoch)),
                )

                os.makedirs(test_results_path, exist_ok=True)

                test_results_accuracy_mc_path = os.path.join(
                    test_results_path,
                    "accuracy_mc.txt"
                )

                test_results_accuracy_non_mc_path = os.path.join(
                    test_results_path,
                    "accuracy_non_mc.txt"
                )

                test_results_lls_mc_path = os.path.join(
                    test_results_path,
                    "lls_mc.txt"
                )

                if len(metrics) > 0:
                    for key, value in metrics.items():
                        print(str(key) + " = " + str(value))

                        if key == 'accuracy_mc':
                            with open(test_results_accuracy_mc_path, 'a+') as accuracy_mc_file:
                                accuracy_mc_file.write('%d %f \n' % (s, value))

                        elif key == 'accuracy_non_mc':
                            with open(test_results_accuracy_non_mc_path, 'a+') as accuracy_non_mc_file:
                                accuracy_non_mc_file.write('%d %f \n' % (s, value))

                        elif key == 'test_ll_mc':
                            with open(test_results_lls_mc_path, 'a+') as lls_mc_file:
                                lls_mc_file.write('%d %f \n' % (s, value))

                print()

Files already downloaded and verified
subset size = (2500, 32, 32, 3)
training set size = 2000
test set size = 500

subset 0.050000, dropout_rate 0.100000, length_scale 0.010000, tau 0.100000
reg_strength = tensor(2.2500e-07)
Training with split 0
Training for 40 epochs total.
epoch 0 loss = 2.275429
epoch 1 loss = 2.206273
epoch 2 loss = 2.140935
epoch 3 loss = 2.069733
epoch 4 loss = 1.998701
epoch 5 loss = 1.993923
epoch 6 loss = 1.941433
epoch 7 loss = 1.926011
epoch 8 loss = 1.888519
epoch 9 loss = 1.838508
epoch 10 loss = 1.833360
epoch 11 loss = 1.781785
epoch 12 loss = 1.747734
epoch 13 loss = 1.708642
epoch 14 loss = 1.698760
epoch 15 loss = 1.651852
epoch 16 loss = 1.664166
epoch 17 loss = 1.608077
epoch 18 loss = 1.595061
epoch 19 loss = 1.554573
epoch 20 loss = 1.519203
epoch 21 loss = 1.474406
epoch 22 loss = 1.459662
epoch 23 loss = 1.433001
epoch 24 loss = 1.435135
epoch 25 loss = 1.413993
epoch 26 loss = 1.392002
epoch 27 loss = 1.386918
epoch 28 loss = 1.292635
epoch 2

epoch 261 loss = 0.000863
epoch 262 loss = 0.000856
epoch 263 loss = 0.000848
epoch 264 loss = 0.000841
epoch 265 loss = 0.000834
epoch 266 loss = 0.000827
epoch 267 loss = 0.000820
epoch 268 loss = 0.000813
epoch 269 loss = 0.000806
epoch 270 loss = 0.000800
epoch 271 loss = 0.000794
epoch 272 loss = 0.000787
epoch 273 loss = 0.000781
epoch 274 loss = 0.000774
epoch 275 loss = 0.000768
epoch 276 loss = 0.000762
epoch 277 loss = 0.000756
epoch 278 loss = 0.000749
epoch 279 loss = 0.000744
epoch 280 loss = 0.000738
epoch 281 loss = 0.000731
epoch 282 loss = 0.000726
epoch 283 loss = 0.000720
epoch 284 loss = 0.000714
epoch 285 loss = 0.000709
epoch 286 loss = 0.000703
epoch 287 loss = 0.000698
epoch 288 loss = 0.000692
epoch 289 loss = 0.000687
epoch 290 loss = 0.000682
epoch 291 loss = 0.000676
epoch 292 loss = 0.000671
epoch 293 loss = 0.000666
epoch 294 loss = 0.000661
epoch 295 loss = 0.000656
epoch 296 loss = 0.000651
epoch 297 loss = 0.000646
epoch 298 loss = 0.000641
epoch 299 lo

epoch 211 loss = 0.000143
epoch 212 loss = 0.000142
epoch 213 loss = 0.000142
epoch 214 loss = 0.000141
epoch 215 loss = 0.000140
epoch 216 loss = 0.000140
epoch 217 loss = 0.000139
epoch 218 loss = 0.000139
epoch 219 loss = 0.000138
epoch 220 loss = 0.000137
epoch 221 loss = 0.000137
epoch 222 loss = 0.000136
epoch 223 loss = 0.000136
epoch 224 loss = 0.000135
epoch 225 loss = 0.000135
epoch 226 loss = 0.000134
epoch 227 loss = 0.000133
epoch 228 loss = 0.000133
epoch 229 loss = 0.000132
epoch 230 loss = 0.000132
epoch 231 loss = 0.000131
epoch 232 loss = 0.000131
epoch 233 loss = 0.000130
epoch 234 loss = 0.000130
epoch 235 loss = 0.000129
epoch 236 loss = 0.000129
epoch 237 loss = 0.000128
epoch 238 loss = 0.000127
epoch 239 loss = 0.000127
epoch 240 loss = 0.000126
epoch 241 loss = 0.000126
epoch 242 loss = 0.000125
epoch 243 loss = 0.000125
epoch 244 loss = 0.000124
epoch 245 loss = 0.000124
epoch 246 loss = 0.000123
epoch 247 loss = 0.000123
epoch 248 loss = 0.000122
epoch 249 lo

epoch 527 loss = 0.000046
epoch 528 loss = 0.000045
epoch 529 loss = 0.000045
epoch 530 loss = 0.000045
epoch 531 loss = 0.000045
epoch 532 loss = 0.000045
epoch 533 loss = 0.000045
epoch 534 loss = 0.000045
epoch 535 loss = 0.000045
epoch 536 loss = 0.000044
epoch 537 loss = 0.000044
epoch 538 loss = 0.000044
epoch 539 loss = 0.000044
epoch 540 loss = 0.000044
epoch 541 loss = 0.000044
epoch 542 loss = 0.000044
epoch 543 loss = 0.000043
epoch 544 loss = 0.000043
epoch 545 loss = 0.000043
epoch 546 loss = 0.000043
epoch 547 loss = 0.000043
epoch 548 loss = 0.000043
epoch 549 loss = 0.000043
epoch 550 loss = 0.000042
epoch 551 loss = 0.000042
epoch 552 loss = 0.000042
epoch 553 loss = 0.000042
epoch 554 loss = 0.000042
epoch 555 loss = 0.000042
epoch 556 loss = 0.000042
epoch 557 loss = 0.000042
epoch 558 loss = 0.000041
epoch 559 loss = 0.000041
epoch 560 loss = 0.000041
epoch 561 loss = 0.000041
epoch 562 loss = 0.000041
epoch 563 loss = 0.000041
epoch 564 loss = 0.000041
epoch 565 lo

## Results visualization

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

experiment_root_directory = os.path.join(
    './test_results',
    'error_convergence_2',
    'CIFAR-10',
    test_start_time,
)

for subset_prop, dropout_rate, length_scale, tau, n_epoch in itertools.product(
    subset_proportions, dropout_rates, length_scale_values, tau_values, n_epochs
):
    for metric_name in ['lls_mc', 'accuracy_mc', 'accuracy_non_mc']:
        figure_file_name = (
            '3d_' + str(subset_prop)
            + '_' + str(dropout_rate)
            + '_' + str(length_scale)
            + '_' + str(tau)
            + '_' + str(n_epoch)
            + '_' + metric_name + '.png'
        )
        
        figure_title = (
            metric_name 
            + (' subset %f, dropout rate = %f, length_scale %f, tau %f, n_epoch = %d' 
               % (subset_prop, dropout_rate, length_scale, tau, n_epoch))
        )

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        fig.tight_layout(pad=2, rect=[0, 0.00, 2, 2])
        
        hidden_dims_data = []
        hidden_layers_data = []
        scores_data = []
        
        for hidden_dim in network_hidden_dims:
            for n_layer in network_hidden_layers:
                # Open the score file
                score_file_path = os.path.join(
                    experiment_root_directory,
                    (
                        str(subset_prop) 
                        + '_' + str(hidden_dim)
                        + '_' + str(n_layer) 
                        + '_' + str(dropout_rate) 
                        + '_' + str(length_scale)
                        + '_' + str(tau)
                        + '_' + str(n_epoch)
                    ),
                    metric_name + '.txt',
                )

                scores = np.loadtxt(score_file_path).T
                
                for s in scores[1]:
                    # Multiple values (for each split) for
                    # each (hidden_dim, n_layer) combination
                    hidden_dims_data.append(hidden_dim)
                    hidden_layers_data.append(n_layer)
                    scores_data.append(s)
                    
                mean = np.mean(scores[1])
                var = np.var(scores[1])

        ax.set_xlabel('hidden layer units')
        ax.set_ylabel('number of hidden layers')
        
        #if metric_name in ('rmse_mc', 'rmse_non_mc'):
        #    ax.set_zlim([5, 20])
        #elif metric_name == 'lls_mc':
        #    ax.set_zlim([-10, 0])
    
        ax.scatter3D(hidden_dims_data, hidden_layers_data, scores_data, c=scores_data)

        fig.suptitle(figure_title, y=2.05)        
        
        plt.savefig(
            os.path.join(experiment_root_directory, figure_file_name),
            dpi=600,
            bbox_inches='tight',
        )
        
        plt.show()